<a href="https://colab.research.google.com/github/kswamy15/langchain_experiments/blob/main/Langchain_sql_query_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the Libraries

In [ ]:
!pip install langchain langchain_experimental
!pip3 install openai

# Download sample SQlite databases

In [ ]:
# download Northwind SQLite DB
!wget https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3

--2023-08-06 18:32:50--  https://tdmdal.github.io/mma-sql-2021/data/northwind.sqlite3
Resolving tdmdal.github.io (tdmdal.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to tdmdal.github.io (tdmdal.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804864 (786K) [application/octet-stream]
Saving to: ‘northwind.sqlite3’

northwind.sqlite3   100%[===================>] 786.00K  --.-KB/s    in 0.009s  

2023-08-06 18:32:50 (83.2 MB/s) - ‘northwind.sqlite3’ saved [804864/804864]



In [ ]:
### useful: download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

downloading chinook.zip ...........................................................................

# Import the langchain libraries

In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

# Initialize the SQL Database Chain

In [ ]:
db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = OpenAI(temperature=0, verbose=True)

In [ ]:
#print(db.table_info)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

# Query OpenAI with questions around the tables in the Database

In [ ]:
db_chain.run("How many albums by Aerosmith?")



> Entering new SQLDatabaseChain chain...
How many albums by Aerosmith?
SQLQuery:SELECT COUNT(*) FROM albums WHERE "ArtistId" IN (SELECT "ArtistId" FROM artists WHERE "Name" = 'Aerosmith');
SQLResult: [(1,)]
Answer:There is 1 album by Aerosmith.
> Finished chain.


'There is 1 album by Aerosmith.'

# Custom Prompt to make the OpenAI properly structure the query

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run,
all column names in the query should be fully qualified by the table they come from, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, use_query_checker=True)

In [ ]:
db_chain.run("Who are the top 10 highest selling artists?")



> Entering new SQLDatabaseChain chain...
Who are the top 10 highest selling artists?
SQLQuery:SELECT artists.Name, SUM(invoice_items.Quantity) AS TotalSales
FROM artists
INNER JOIN albums ON albums.ArtistId = artists.ArtistId
INNER JOIN tracks ON tracks.AlbumId = albums.AlbumId
INNER JOIN invoice_items ON invoice_items.TrackId = tracks.TrackId
GROUP BY artists.Name
ORDER BY TotalSales DESC
LIMIT 10;
SQLResult: [('Iron Maiden', 140), ('U2', 107), ('Metallica', 91), ('Led Zeppelin', 87), ('Os Paralamas Do Sucesso', 45), ('Deep Purple', 44), ('Faith No More', 42), ('Lost', 41), ('Eric Clapton', 40), ('R.E.M.', 39)]
Answer:The top 10 highest selling artists are Iron Maiden, U2, Metallica, Led Zeppelin, Os Paralamas Do Sucesso, Deep Purple, Faith No More, Lost, Eric Clapton, and R.E.M.
> Finished chain.


'The top 10 highest selling artists are Iron Maiden, U2, Metallica, Led Zeppelin, Os Paralamas Do Sucesso, Deep Purple, Faith No More, Lost, Eric Clapton, and R.E.M.'

In [ ]:
db_chain.run("print the names of top 10 tracks sold, and how many times they were sold in a html format")



> Entering new SQLDatabaseChain chain...
print the names of top 10 tracks sold, and how many they times they were sold in a html format
SQLQuery:SELECT tracks.Name, COUNT(invoice_items.TrackId) AS 'Number Sold'
FROM tracks
INNER JOIN invoice_items
ON tracks.TrackId = invoice_items.TrackId
GROUP BY tracks.Name
ORDER BY COUNT(invoice_items.TrackId) DESC
LIMIT 10;
SQLResult: [('The Trooper', 5), ('Untitled', 4), ('The Number Of The Beast', 4), ('Sure Know Something', 4), ('Hallowed Be Thy Name', 4), ('Eruption', 4), ('Where Eagles Dare', 3), ('Welcome Home (Sanitarium)', 3), ('Sweetest Thing', 3), ('Surrender', 3)]
Answer:The top 10 tracks sold are The Trooper (5), Untitled (4), The Number Of The Beast (4), Sure Know Something (4), Hallowed Be Thy Name (4), Eruption (4), Where Eagles Dare (3), Welcome Home (Sanitarium) (3), Sweetest Thing (3), and Surrender (3).
> Finished chain.


'The top 10 tracks sold are The Trooper (5), Untitled (4), The Number Of The Beast (4), Sure Know Something (4), Hallowed Be Thy Name (4), Eruption (4), Where Eagles Dare (3), Welcome Home (Sanitarium) (3), Sweetest Thing (3), and Surrender (3).'